In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

# 1. Charger les données historiques du Bitcoin
df = pd.read_csv('Bitcoin_Historical_Data.csv', parse_dates=['Date'], index_col='Date')

# 2. Prétraiter les données
window_size = 30
data = df.values
sequence_length = window_size + 1
result = []
for index in range(len(data) - sequence_length):
    result.append(data[index : index + sequence_length])
result = np.array(result)

# 3. Normaliser les données
scaler = MinMaxScaler(feature_range=(0, 1))
result = scaler.fit_transform(result.reshape(-1, 1)).reshape(-1, sequence_length)

# 4. Diviser les données en ensembles d'entraînement et de test
row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
np.random.shuffle(train)
x_train = train[:, :-1]
y_train = train[:, -1]
x_test = result[row:, :-1]
y_test = result[row:, -1]

# 5. Définir un modèle de RNN avec des couches LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(window_size,


In [ ]:
    # 5. Définir un modèle de RNN avec des couches LSTM
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(window_size, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    # 6. Compiler le modèle
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 7. Entraîner le modèle
    model.fit(x_train, y_train, epochs=50, batch_size=32)

    # 8. Évaluer les performances du modèle
    train_predict = model.predict(x_train)
    test_predict = model.predict(x_test)

    # Convertir les prédictions normalisées en valeurs réelles
    train_predict = scaler.inverse_transform(train_predict)
    y_train = scaler.inverse_transform([y_train])
    test_predict = scaler.inverse_transform(test_predict)
    y_test = scaler.inverse_transform([y_test])

    # Calculer les métriques de performance
    train_rmse = np.sqrt(np.mean((train_predict - y_train)**2))
    test_rmse = np.sqrt(np.mean((test_predict - y_test)**2))
    print('RMSE (train): {:.2f}'.format(train_rmse))
    print('RMSE (test): {:.2f}'.format(test_rmse))

    # 9. Faire des prévisions
    days = 30
    future = []
    last_x = x_test[-1]
    for i in range(days):
        prediction = model.predict(last_x.reshape(1, window_size, 1))
        future.append(prediction)
        last_x = np.append(last_x[1:], prediction)
    future = scaler.inverse_transform(future)

    # Visualiser les résultats
    plt.plot(df.index, df['Price'], label='Données réelles')
    plt.plot(df.index[-len(train_predict):], train_predict, label='Prévisions sur les données d\'entraînement')
    plt.plot(df.index[-len(test_predict):], test_predict, label='Prévisions sur les données de test')
    plt.plot(pd.date_range(start=df.index[-1], periods=days + 1, freq='D')[1:], future, label='Prévisions futures')
    plt.legend()
    plt.show()
